In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
graph = Graph(password="password")

query = "match (n) return head(labels(n)) as label, count(*);"
result = graph.run(query)

print(result.to_data_frame())

   label  count(*)
0   Post     39929
1   User     66951
2    Tag       489
3  Stack      6187


#### Create constraints and indices - easier in browser - localhost:7474

In [3]:
# create index on :Post(title);
# create index on :Post(createdAt);
# create index on :Post(score);
# create index on :Post(views);
# create index on :Post(favorites);
# create index on :Post(answers);
# create index on :Post(score);

# create index on :User(name);
# create index on :User(createdAt);
# create index on :User(reputation);
# create index on :User(age);

# create index on :Tag(count);

# create constraint on (t:Tag) assert t.tagId is unique;
# create constraint on (u:User) assert u.userId is unique;
# create constraint on (p:Post) assert p.postId is unique;

# We then waited for the indexes to be finished.

# schema await

In [4]:
query = "match (u:User) with u,size( (u)-[:POSTED]->()) as posts order by posts desc limit 10 return u.displayname, posts;" 
result = graph.run(query)

print(result.to_data_frame())

    u.displayname  posts
0     Neil Slater    403
1     Vaalizaadeh    354
2         n1k31t4    347
3    Anony-Mousse    325
4  Brian Spiering    241
5            Emre    227
6    Martin Thoma    224
7         Dawny33    215
8           Tasos    205
9        JahKnows    203


#### Get number of tags used in asking questions

In [5]:
query = "match (u:User)-[:POSTED]->()-[:HAS_TAG]->(t:Tag) where u.displayname = 'Vaalizaadeh'  return t,count(*) as posts order by posts desc limit 5;" 
result = graph.run(query)

print(result.to_data_frame())

# This user only answered questions

                               t  posts
0  {'tagId': 'machine-learning'}     20
1     {'tagId': 'deep-learning'}     15
2    {'tagId': 'neural-network'}     11
3               {'tagId': 'cnn'}      4
4             {'tagId': 'keras'}      2


#### Get tags that User answered

In [6]:
query = "match (u:User)-[:POSTED]->()-[:HAS_TAG]->(t:Tag) where u.displayname = 'Puffin GDI' return t.tagId,count(*) as posts order by posts desc limit 5;" 
result = graph.run(query)

print(result.to_data_frame())

            t.tagId  posts
0           bigdata      1
1            libsvm      1
2  machine-learning      1


#### All shortest paths - run in neo4j console or browser

In [7]:
# MATCH path = allShortestPaths(
#      (u:User {displayname:"Emre"})-[*]-(me:User {displayname:"Vaalizaadeh"}))
# RETURN path;

![title](images/all_short_path.png)

#### User with letter t in name answered the most questions about python

In [45]:
query = "MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag {tagId:'python'}) WHERE u.displayname contains 't' RETURN u.displayname AS name,u.reputation AS rep,u.location, count(distinct answer) AS answers ORDER BY answers DESC LIMIT 20;" 
result = graph.run(query)
#print(result.to_data_frame())
xo = result.to_data_frame()
xo["rep/ans"] = xo["rep"].astype(float) / xo["answers"].astype(float)
#print(xo)
xo.head(40)

# query = "" 
# result = graph.run(query)
# print(result.to_data_frame())

,name,rep,u.location,answers,rep/ans
0,n1k31t4,8181,Europa,117,69.923077
1,Neil Slater,18824,"Durham, United Kingdom",36,522.888889
2,Juan Esteban de la Calle,1504,"Bogotá, Colombia",24,62.666667
3,Jan van der Vegt,6995,Amsterdam,23,304.130435
4,Aditya,1577,"West Bengal, India",22,71.681818
5,Stephen Rauch,1598,"Northern California, United States",19,84.105263
6,Peter,1648,"Frankfurt am Main, Deutschland",17,96.941176
7,Ankit Seth,1159,None,17,68.176471
8,Kiritee Gak,1379,"Delhi, India",15,91.933333
9,Shamit Verma,1754,"Mumbai, India",14,125.285714


In [46]:
import plotly_express as px         
px.bar(xo, x='name', y='rep')

In [47]:
fig = px.scatter(xo, x="answers", y="rep", color="name", size="rep/ans", log_x=True, log_y=True)
fig.show()

In [47]:
# MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag {tagId:"nosql"}) 
# WHERE u.displayname = 'Emre'
# RETURN u.displayname, u.reputation,u.location, count(distinct answer) AS answers
# ORDER BY answers DESC;

#### Run this graph in browser/ console

In [62]:
# MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-(question)-[:HAS_TAG]-(tag:Tag {tagId:"python"}) 
# WHERE u.displayname contains "t"
# RETURN *,count(distinct answer) AS answers
# ORDER BY answers DESC limit 30;

![title](images/python_answers.png)

#### Top 5 tags of all time

In [73]:
query = "match (t:Tag) with t order by t.count desc limit 5 return t.tagId, t.count;" 
result = graph.run(query)
print(result.to_data_frame())

            t.tagId t.count
0  machine-learning    None
1           bigdata    None
2       data-mining    None
3         databases    None
4       definitions    None


#### Co-occurence with 'javascript' tag

In [75]:
query = "match (t:Tag {tagId:'javascript'})<-[:HAS_TAG]-()-[:HAS_TAG]->(other:Tag) WITH other, count(*) as freq order by freq desc limit 5 RETURN other.tagId,freq;" 
result = graph.run(query)
print(result.to_data_frame())

# match (t:Tag {tagId:"javascript"})<-[:HAS_TAG]-()-[:HAS_TAG]->(other:Tag) 
# WITH other, count(*) as freq order by freq desc limit 5
# RETURN other.tagId,freq;

        other.tagId  freq
0     visualization    11
1    neural-network     6
2  machine-learning     6
3            python     3
4            graphs     2


#### Most active answerers for python

In [79]:
query = "match (t:Tag {tagId:'python'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) WITH u, count(*) as freq order by freq desc limit 10 RETURN u.displayname,freq;" 
result = graph.run(query)
print(result.to_data_frame())


# match (t:Tag {tagId:"neo4j"})<-[:HAS_TAG]-()
#        -[:PARENT_OF]->()<-[:POSTED]-(u:User) 
# WITH u, count(*) as freq order by freq desc limit 10
# RETURN u.name,freq;

    u.displayname  freq
0         n1k31t4   117
1        JahKnows    52
2  Brian Spiering    49
3    Anony-Mousse    49
4            Emre    42
5     Vaalizaadeh    38
6     Neil Slater    36
7            MaxU    34
8  Kasra Manshaei    31
9           Tasos    28


#### Where are the most active answerers also active

In [82]:
query = "MATCH (neo:Tag {tagId:'python'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) WITH neo,u, count(*) as freq order by freq desc limit 10 MATCH (u)-[:POSTED]->()<-[:PARENT_OF]-(p)-[:HAS_TAG]->(other:Tag) WHERE NOT (p)-[:HAS_TAG]->(neo) WITH u,other,count(*) as freq2 order by freq2 desc RETURN u.displayname,collect(distinct other.tagId)[1..5] as tags;" 
result = graph.run(query)
print(result.to_data_frame())


# MATCH (neo:Tag {tagId:'neo4j'})<-[:HAS_TAG]-()
#       -[:PARENT_OF]->()<-[:POSTED]-(u:User) 
# WITH neo,u, count(*) as freq order by freq desc limit 10
# MATCH (u)-[:POSTED]->()<-[:PARENT_OF]-(p)-[:HAS_TAG]->(other:Tag)
# WHERE NOT (p)-[:HAS_TAG]->(neo)
# WITH u,other,count(*) as freq2 order by freq2 desc 
# RETURN u.displayname,collect(distinct other.tagId)[1..5] as tags;

    u.displayname                                               tags
0     Vaalizaadeh        [deep-learning, neural-network, cnn, keras]
1    Anony-Mousse  [machine-learning, k-means, data-mining, class...
2     Neil Slater  [neural-network, reinforcement-learning, deep-...
3            Emre  [neural-network, classification, deep-learning...
4         n1k31t4  [deep-learning, neural-network, keras, tensorf...
5        JahKnows  [neural-network, deep-learning, keras, classif...
6  Brian Spiering  [nlp, deep-learning, neural-network, text-mining]
7  Kasra Manshaei  [clustering, graphs, classification, data-mining]
8           Tasos             [pandas, nlp, statistics, time-series]


#### Posted most questions about neo4j

In [85]:
query = "MATCH (t:Tag {tagId:'neo4j'})<-[:HAS_TAG]-(:Post)<-[:POSTED]-(u:User) RETURN u.displayname,count(*) as count ORDER BY count DESC LIMIT 10;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (t:Tag {tagId:'neo4j'})<-[:HAS_TAG]-(:Post)<-[:POSTED]-(u:User)
# RETURN u.name,count(*) as count
# ORDER BY count DESC LIMIT 10;

       u.displayname  count
0        John Newman      2
1       Sreejithc321      2
2        TobiasJakob      1
3           blunders      1
4                SRS      1
5      sheldonkreger      1
6             Victor      1
7  Austin Capobianco      1


#### Top answerers for pandas tag

In [88]:
query = "MATCH (:Tag {tagId:'pandas'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) RETURN u.displayname,count(*) as count ORDER BY count DESC LIMIT 10;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (:Tag {tagId:'py2neo'})<-[:HAS_TAG]-()-[:PARENT_OF]->()
#       <-[:POSTED]-(u:User)
# RETURN u.displayname,count(*) as count
# ORDER BY count DESC LIMIT 10;

   u.displayname  count
0        n1k31t4     36
1           MaxU     24
2           Emre     18
3          Tasos     15
4         Aditya     12
5          bkshi     12
6  Stephen Rauch     10
7    Vaalizaadeh     10
8   TwinPenguins      8
9       JahKnows      8


#### Users who answered their own question 

In [94]:
query = "MATCH (u:User) WHERE u.reputation > 1000 MATCH (u)-[:POSTED]->(question)-[:ANSWER]->(answer)<-[:POSTED]-(u) WITH u,count(distinct question) AS questions ORDER BY questions DESC LIMIT 5 RETURN u.displayname, u.reputation, questions;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (u:User) WHERE u.reputation > 20000
# MATCH (u)-[:POSTED]->(question)-[:ANSWER]->(answer)<-[:POSTED]-(u)
# WITH u,count(distinct question) AS questions
# ORDER BY questions DESC LIMIT 5
# RETURN u.displayname, u.reputation, questions;

# not working no :ANSWER relationship

Empty DataFrame
Columns: []
Index: []


#### Smaller queries

In [ ]:
# MATCH path = allShortestPaths( (t:Tag {tagId:"activation"})-[*]-(u:Tag {tagId:"xboost"})) RETURN path LIMIT 10;

![title](images/xboost_activation_bigdb.png)

In [1]:
# MATCH path = allShortestPaths( (t:User {displayname:"black_cat"})-[*]-(u:User {displayname:"Skier"})) RETURN path LIMIT 10;

![title](images/black_cat_to_skier.png)